In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date, datetime, timedelta
from sklearn import linear_model

C:\Users\marie\AppData\Local\Temp\ipykernel_18108\3028730387.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def interval_seconds(interval_str):
    if len(interval_str.split(" d ")) > 1:
        days = int(interval_str.split(" d ")[0])
        times = interval_str.split(" d ")[1].split(":")
    else:
        days = 0
        times = interval_str.split(" d ")[-1].split(":")
    hours = int(times[0])
    minutes = int(times[1])
    seconds = int(times[2])
    interval = timedelta(days = days, hours = hours, minutes = minutes, seconds = seconds).total_seconds()
    return interval

In [3]:
def merge_long(df, info_cols, col_dict):
    for i, key in enumerate(col_dict.keys()):
        sel_cols = info_cols + col_dict[key]
        temp = df[sel_cols]
        # select columns and melt 
        temp2 = pd.wide_to_long(temp, [key], i = info_cols, j="teat", sep='_', suffix=r'\w+').reset_index()
        if i == 0:
            df_long = temp2.copy()
        else:
            df_long = pd.merge(df_long, temp2, on = info_cols + ["teat"])
    return df_long

def get_col_dict(df, teat_list):
    # find columns with teats
    columns_q = [col for col in df.columns if col.split("_")[-1] in teat_list]
    col_dict = {}
    for col in  columns_q:
        key = "_".join(col.split("_")[0:-1])
        if key in col_dict.keys():
            col_dict[key].append(col)
        else:
            col_dict[key] = [col]
    return col_dict

In [10]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "..", "delpro_vms", "data")
feature_path = os.path.join(dirpath, "..", "udder_processing", "features_dict")
out_dir = "long_format_df"
# read vistit data
visit_df = pd.read_csv(os.path.join(data_path, "milk_videos_visit.csv"))
visit_df["interval_sec"] = [interval_seconds(rec) for rec in visit_df.interval]
visit_df = visit_df.drop_duplicates(subset='cow', keep="first")
# read features
feature_df = pd.read_csv(os.path.join(feature_path, "gmfeature_table.csv"))
feature_df = feature_df.rename(columns = lambda col: "_".join(col.split("_")[::-1]) if "_" in col else col)

In [5]:
info_cols = ['cow', 'date', 'begin_time', 'days_in_milk', 'device_name', 'is_incomplete', 'interval_sec']
col_dict = get_col_dict(visit_df, ["lf", "lr", "rr", "rf"])
visit_long = merge_long(visit_df, info_cols, col_dict)
# feature long format
info_cols = ["cow"]
col_dict = get_col_dict(feature_df, ["lf", "lb", "rb", "rf"])
ft_long = merge_long(feature_df, info_cols, col_dict)
ft_long["teat"] = [t.replace("b", "r") for t in ft_long.teat]

merged_long = pd.merge(ft_long, visit_long, left_on = ["cow", "teat"], right_on = ["cow", "teat"])
merged_long.to_csv(os.path.join(out_dir, "visit_ft_long.csv"), index = False)

In [9]:
info_cols = ['cow', 'date', 'begin_time', 'days_in_milk', 'device_name', 'is_incomplete', 'interval_sec']
col_dict = ['yield', 'is_incomplete', 'occ']
visit_long = visit_df[info_cols + col_dict]
# feature long format
info_cols = ["cow"]
col_dict =  [col for col in feature_df.columns if col.split("_")[-1] == "udder"]
ft_long = feature_df[info_cols+col_dict]

merged_long = pd.merge(ft_long, visit_long, left_on = ["cow"], right_on = ["cow"])
merged_long.to_csv(os.path.join(out_dir, "visit_ft_long_udder.csv"), index = False)